In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.7 MB/s eta 0:00:00


In [2]:
import os # used to work with the operating system and terminal commnands
import pprint #use to print the interpreter output
import tempfile # Used to create temporary files and folders

from typing import Dict, Text #These are datatypes

import numpy as np # For mathematical computations
import tensorflow as tf # The Deep learning Library
import tensorflow_datasets as tfds # The library to import data

In [3]:
import tensorflow_recommenders as tfrs #tfrs = tensorflow recommedation system

In [4]:
'''
We will be using the TF dataset by the name of movielens which has
ratings of various movies, this dataset is not pre-divided into splits
'''
#Import the Rating data
ratings = tfds.load("movielens/100k-ratings",split='train')
#Import all the features data for these ratings
movies = tfds.load("movielens/100k-movies",split='train')


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompletePP2HUV/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [5]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [6]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [7]:
ratings = ratings.map(lambda x: {
    "movie_title":x['movie_title'],
    "user_id": x["user_id"]
})

movies = movies.map(lambda x:x["movie_title"])

In [8]:
ratings_sample = None
for x in ratings:
  ratings_sample = x
  break


print(f"Type of ratings data set = {type(ratings)}\n")
print(f"The len of ratings dataset = {len(ratings)}\n")
print(f"The structure of data in ratings is of 2 keys\n1.movie_title\n2.User_id\n\nsample={ratings_sample}")

Type of ratings data set = <class 'tensorflow.python.data.ops.map_op._MapDataset'>

The len of ratings dataset = 100000

The structure of data in ratings is of 2 keys
1.movie_title
2.User_id

sample={'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>}


In [9]:
movies_sample = None
iter= 0 # I am adding this iter variable to store different index value in my sample
for x in ratings:
    if iter < 3:
      movies_sample = x
      iter+=1
    else:
      break


print(f"Type of ratings data set = {type(movies)}\n")
print(f"The len of ratings dataset = {len(movies)}\n")
print(f"The structure of data in movies is of 1 key\n1.movie_title\n\nsample={movies_sample}")

Type of ratings data set = <class 'tensorflow.python.data.ops.map_op._MapDataset'>

The len of ratings dataset = 1682

The structure of data in movies is of 1 key
1.movie_title

sample={'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Very Brady Sequel, A (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'301'>}


In [10]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [11]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x:x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [12]:
unique_user_ids[0:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

In [13]:
unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

*THE QUERY TOWER*

In [14]:
embedding_dimension = 32

In [15]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [16]:
movie_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
        tf.keras.layers.Embedding(len(unique_movie_titles)+1,
                                  embedding_dimension
    )
])

In [17]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128).map(movie_model)
)

In [18]:
#Loss Evaluation
task = tfrs.tasks.Retrieval(
    metrics=metrics
)

In [19]:
# WE can use the above parts of the model and make a fully
# functional model

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [20]:
#Lets fit this Movielens model
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


In [21]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [22]:
model.fit(cached_train, epochs=8)

Epoch 1/8
10/10 [==============================] - 34s 3s/step - factorized_top_k/top_1_categorical_accuracy: 9.7500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0088 - factorized_top_k/top_10_categorical_accuracy: 0.0195 - factorized_top_k/top_50_categorical_accuracy: 0.0987 - factorized_top_k/top_100_categorical_accuracy: 0.1756 - loss: 69832.6065 - regularization_loss: 0.0000e+00 - total_loss: 69832.6065
Epoch 2/8
10/10 [==============================] - 28s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0030 - factorized_top_k/top_5_categorical_accuracy: 0.0192 - factorized_top_k/top_10_categorical_accuracy: 0.0381 - factorized_top_k/top_50_categorical_accuracy: 0.1676 - factorized_top_k/top_100_categorical_accuracy: 0.2897 - loss: 67450.7756 - regularization_loss: 0.0000e+00 - total_loss: 67450.7756
Epoch 3/8
10/10 [==============================] - 34s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0034 - factorized_top_k/top_5_categorical_accuracy: 0.

In [23]:
#Evaluate the model on test data
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 9s 999ms/step - factorized_top_k/top_1_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0033 - factorized_top_k/top_10_categorical_accuracy: 0.0103 - factorized_top_k/top_50_categorical_accuracy: 0.0910 - factorized_top_k/top_100_categorical_accuracy: 0.1966 - loss: 31480.7480 - regularization_loss: 0.0000e+00 - total_loss: 31480.7480


{'factorized_top_k/top_1_categorical_accuracy': 0.0003499999875202775,
 'factorized_top_k/top_5_categorical_accuracy': 0.0032500000670552254,
 'factorized_top_k/top_10_categorical_accuracy': 0.010250000283122063,
 'factorized_top_k/top_50_categorical_accuracy': 0.09104999899864197,
 'factorized_top_k/top_100_categorical_accuracy': 0.19664999842643738,
 'loss': 28576.626953125,
 'regularization_loss': 0,
 'total_loss': 28576.626953125}

In [24]:
# Lets create a prediction method

#Let's first create a model that takes in raw query and recommends movie from the
# available dataset
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

index.index_from_dataset(
    tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

#Get Recommendations
_, titles = index(tf.constant(["42"]))
print(f"Recommendatios for user-42 : {titles[0,:3]}")

Recommendatios for user-42 : [b'Halloween: The Curse of Michael Myers (1995)' b'Unforgettable (1996)'
 b'Rent-a-Kid (1995)']
